### Import and Config

In [2]:
import os

from model_setup import Model_Setup
from PoseDataGenerator import PoseDataGenerator
from Plot3D import Landmark_list,plot_landmarks
from Train import Train

PATH_VIDEO = 'videos'
RESULT_CSV = 'results'

config = Model_Setup()


### Generate training data from Video Clip

In [1]:
pose_data = PoseDataGenerator(config)
if os.path.isfile(RESULT_CSV) == False:
    pose_data.generate_pose_multifile(PATH_VIDEO,RESULT_CSV)

NameError: name 'PoseDataGenerator' is not defined

### Testing 3DPlot Pose Estimation

In [ ]:
landmark_list = Landmark_list(list_size = config.NUM_COORDS)
df = landmark_list.load_csv(RESULT_CSV+'\coordinates_1.csv')

In [ ]:
'''
# Plot every frame
for i in range(len(df)):
    landmark_list = Landmark_list(list_size = 33)
    df_temp = df.iloc[i,:]
    x = df_temp[df.columns.str.startswith('x')].to_numpy().flatten()
    y = df_temp[df.columns.str.startswith('y')].to_numpy().flatten()
    z = df_temp[df.columns.str.startswith('z')].to_numpy().flatten()
    visibility = df_temp[df.columns.str.startswith('v')].to_numpy().flatten()
    landmark_list.load_xyz(x,y,z,visibility)
    plot_landmarks(landmark_list, pose_data.mp_pose.POSE_CONNECTIONS)
'''

###  Train 


In [3]:
config.MODEL = "autoencoder"
train_obj = Train(config)

train_obj.fit(RESULT_CSV)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 72, 99)]     0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 72, 100), (N 80000       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 100), (None, 80400       lstm[0][0]                       
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 1, 100)       0           lstm_1[0][0]                     
______________________________________________________________________________________________

KeyboardInterrupt: 

In [ ]:
train_obj.plot_performance()

In [ ]:
import numpy as np
X = []

Y = []
a = [[0,0,0],[1,2,3],[10,20,30]]
b = [[4,5,6],[40,50,60]]
X.append(a)
X.append(b)

#X = np.array(X)
#Y = np.array(Y)

X

In [ ]:
X[0][0]

In [ ]:
X.shape

In [ ]:
X.reshape((2,-1))
X

In [ ]:
X.shape

In [ ]:
X = np.concatenate((X,Y),axis=0)
X.shape

In [ ]:
y